## Question 1: Read the given CSV file in a Hive table

In [1]:
#!pip install pyspark

In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName("MLhands-on").enableHiveSupport().getOrCreate()

## Question 2: Read the data from Hive table as spark dataframe

### Reading the dataset

In [5]:
df=spark.read.csv('boston.csv',header='true', inferSchema='true')

checking cache

In [6]:
df.cache()

DataFrame[CRIM: double, ZN: double, INDUS: double, CHAS: int, NOX: double, RM: double, AGE: double, DIS: double, RAD: int, TAX: int, PT: double, B: double, LSTAT: double, MV: double]

PrintSchema()

In [7]:
df.printSchema()

root
 |-- CRIM: double (nullable = true)
 |-- ZN: double (nullable = true)
 |-- INDUS: double (nullable = true)
 |-- CHAS: integer (nullable = true)
 |-- NOX: double (nullable = true)
 |-- RM: double (nullable = true)
 |-- AGE: double (nullable = true)
 |-- DIS: double (nullable = true)
 |-- RAD: integer (nullable = true)
 |-- TAX: integer (nullable = true)
 |-- PT: double (nullable = true)
 |-- B: double (nullable = true)
 |-- LSTAT: double (nullable = true)
 |-- MV: double (nullable = true)



In [8]:
#Reading the dataset 
df.show(5)

+-----------+----+-----------+----+-----------+-----------+-----------+-----------+---+---+-----------+-----------+-----------+-----------+
|       CRIM|  ZN|      INDUS|CHAS|        NOX|         RM|        AGE|        DIS|RAD|TAX|         PT|          B|      LSTAT|         MV|
+-----------+----+-----------+----+-----------+-----------+-----------+-----------+---+---+-----------+-----------+-----------+-----------+
|    0.00632|18.0|2.309999943|   0|0.537999988|6.574999809|65.19999695|4.090000153|  1|296|15.30000019|396.8999939|4.980000019|       24.0|
|0.027310001| 0.0|7.070000172|   0|0.469000012|6.421000004|78.90000153|4.967100143|  2|242|17.79999924|396.8999939|9.140000343|21.60000038|
|    0.02729| 0.0|7.070000172|   0|0.469000012|7.184999943|61.09999847|4.967100143|  2|242|17.79999924|392.8299866| 4.03000021|34.70000076|
|0.032370001| 0.0|2.180000067|   0|0.458000004|6.998000145|45.79999924|6.062200069|  3|222|18.70000076|394.6300049|2.940000057|33.40000153|
|0.069049999| 0.0|2.

In [9]:
#Dataset description
df.describe().toPandas()

,summary,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PT,B,LSTAT,MV
0,count,506,506,506,506,506,506,506,506,506,506,506,506,506,506
1,mean,3.6135235608162057,11.363636363636363,11.136778749531626,0.0691699604743083,0.5546950602312246,6.28463438896641,68.57490120115612,3.7950426960059325,9.549407114624506,408.2371541501976,18.45553382776679,356.67402960597883,12.653063233922925,22.53280636250988
2,stddev,8.601545086715594,23.32245299451514,6.86035298095724,0.2539940413404101,0.1158776754570543,0.7026171549511354,28.148861532793276,2.105710142043288,8.707259384239366,168.53711605495903,2.164945780039869,91.29486340272308,7.141061500195388,9.197104107945272
3,min,0.00632,0.0,0.460000008,0,0.38499999,3.561000109,2.900000095,1.129600048,1,187,12.60000038,0.319999993,1.730000019,5.0
4,max,88.97619629,100.0,27.73999977,1,0.870999992,8.779999733,100.0,12.12650013,24,711,22.0,396.8999939,37.97000122,50.0


## Question 3. Get the correlation between dependent and independent variables


In [10]:
pandas_df = df.toPandas()
pandas_df.head(5)

Correlation explains how one or more variables are related to each other. These variables can be input data features which have been used to forecast our target variable.

In [12]:
pandas_df.corr()['MV']

CRIM    -0.388305
ZN       0.360445
INDUS   -0.483725
CHAS     0.175260
NOX     -0.427321
RM       0.695360
AGE     -0.376955
DIS      0.249929
RAD     -0.381626
TAX     -0.468536
PT      -0.507787
B        0.333461
LSTAT   -0.737663
MV       1.000000
Name: MV, dtype: float64

In [13]:
feature_columns = df.columns[:-1]

In [14]:
feature_columns

['CRIM',
 'ZN',
 'INDUS',
 'CHAS',
 'NOX',
 'RM',
 'AGE',
 'DIS',
 'RAD',
 'TAX',
 'PT',
 'B',
 'LSTAT']

## Question 4: Build a linear regression model to predict house price

In [15]:
from pyspark.ml.feature import VectorAssembler

In [16]:
assembler = VectorAssembler(inputCols= feature_columns, outputCol= "features")

In [17]:
vector = assembler.transform(df)

In [18]:
pands_vector = vector.toPandas()
pands_vector.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PT,B,LSTAT,MV,features
0,0.00632,18.0,2.31,0,0.538,6.575,65.199997,4.0900,1,296,15.300000,396.899994,4.98,24.000000,"[0.00632, 18.0, 2.309999943, 0.0, 0.537999988,..."
1,0.02731,0.0,7.07,0,0.469,6.421,78.900002,4.9671,2,242,17.799999,396.899994,9.14,21.600000,"[0.027310001, 0.0, 7.070000172, 0.0, 0.4690000..."
2,0.02729,0.0,7.07,0,0.469,7.185,61.099998,4.9671,2,242,17.799999,392.829987,4.03,34.700001,"[0.02729, 0.0, 7.070000172, 0.0, 0.469000012, ..."
3,0.03237,0.0,2.18,0,0.458,6.998,45.799999,6.0622,3,222,18.700001,394.630005,2.94,33.400002,"[0.032370001, 0.0, 2.180000067, 0.0, 0.4580000..."
4,0.06905,0.0,2.18,0,0.458,7.147,54.200001,6.0622,3,222,18.700001,396.899994,5.33,36.200001,"[0.069049999, 0.0, 2.180000067, 0.0, 0.4580000..."


In [19]:
vector.show()

+-----------+----+-----------+----+-----------+-----------+-----------+-----------+---+---+-----------+-----------+-----------+-----------+--------------------+
|       CRIM|  ZN|      INDUS|CHAS|        NOX|         RM|        AGE|        DIS|RAD|TAX|         PT|          B|      LSTAT|         MV|            features|
+-----------+----+-----------+----+-----------+-----------+-----------+-----------+---+---+-----------+-----------+-----------+-----------+--------------------+
|    0.00632|18.0|2.309999943|   0|0.537999988|6.574999809|65.19999695|4.090000153|  1|296|15.30000019|396.8999939|4.980000019|       24.0|[0.00632,18.0,2.3...|
|0.027310001| 0.0|7.070000172|   0|0.469000012|6.421000004|78.90000153|4.967100143|  2|242|17.79999924|396.8999939|9.140000343|21.60000038|[0.027310001,0.0,...|
|    0.02729| 0.0|7.070000172|   0|0.469000012|7.184999943|61.09999847|4.967100143|  2|242|17.79999924|392.8299866| 4.03000021|34.70000076|[0.02729,0.0,7.07...|
|0.032370001| 0.0|2.180000067|   0

In [20]:
vector_df = vector.select(['features', 'MV'])
vector_df.show()

+--------------------+-----------+
|            features|         MV|
+--------------------+-----------+
|[0.00632,18.0,2.3...|       24.0|
|[0.027310001,0.0,...|21.60000038|
|[0.02729,0.0,7.07...|34.70000076|
|[0.032370001,0.0,...|33.40000153|
|[0.069049999,0.0,...|36.20000076|
|[0.029850001,0.0,...|28.70000076|
|[0.088289998,12.5...|22.89999962|
|[0.144549996,12.5...|27.10000038|
|[0.211239994,12.5...|       16.5|
|[0.170039997,12.5...|18.89999962|
|[0.224889994,12.5...|       15.0|
|[0.117470004,12.5...|18.89999962|
|[0.093780003,12.5...|21.70000076|
|[0.629760027,0.0,...|20.39999962|
|[0.637960017,0.0,...|18.20000076|
|[0.627390027,0.0,...|19.89999962|
|[1.053930044,0.0,...|23.10000038|
|[0.784200013,0.0,...|       17.5|
|[0.802709997,0.0,...|20.20000076|
|[0.725799978,0.0,...|18.20000076|
+--------------------+-----------+
only showing top 20 rows



In [21]:
splits = vector_df.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

In [22]:
from pyspark.ml.regression import LinearRegression

In [23]:
model = LinearRegression(featuresCol= 'features', labelCol= 'MV')
model_lr = model.fit(train_df)    

In [24]:
model_lr.coefficients

DenseVector([-0.1441, 0.051, 0.0069, 3.0814, -18.511, 3.6444, -0.0039, -1.5512, 0.4124, -0.0149, -0.9981, 0.01, -0.5169])

In [25]:
# CRIM - Per Capita crime rate 
# ZN - Proportion of residential land zoned for lots over 25000 sq. ft
# INDUS - Proportion of non-retial business acres 
# CHAS - Charles River dummy variable (1 - if tracts bounds river, 0 -otherwise)
# NOX - Nitrogen Oxide concentration
# RM - Average number of rooms per dwelling
# AGE - Proportion of owner-occupied unit built prior 1940
# DIS - Weighted MEan of distances of five Boston Employement Centres
# RAD - Index of accessibilities to Radial highways
# TAX - Full-value-property-tax rates per $10,000
# PT - Pupil-teacher Ratio
# B - proportion of blacks 
# LSTAT - Lower Status of the Population (%)
# MV - Median Value of homes 

In [26]:
model_lr.intercept

39.267672567791216

In [27]:
predictions = model_lr.transform(test_df) # we use transform method to test the model
predictions.show()

+--------------------+-----------+------------------+
|            features|         MV|        prediction|
+--------------------+-----------+------------------+
|[0.01311,90.0,1.2...|35.40000153|31.553693632343943|
|[0.02055,85.0,0.7...|24.70000076|24.768308541596152|
|[0.02177,82.5,2.0...|42.29999924| 37.06391646804898|
|[0.02187,60.0,2.9...|31.10000038| 32.24792705080656|
|[0.02899,40.0,1.2...|26.60000038|21.499407704123705|
|[0.033059999,0.0,...|20.60000038|22.368885139341153|
|[0.034449998,82.5...|24.10000038| 29.44766870147291|
|[0.035020001,80.0...|       28.5|  33.8149350587754|
|[0.035100002,95.0...|       48.5| 42.57840661736431|
|[0.03548,80.0,3.6...|20.89999962| 21.67021644061087|
|[0.041129999,25.0...|       28.0|28.429351844801612|
|[0.042939998,28.0...|20.60000038| 27.02994546526208|
|[0.043790001,80.0...|19.39999962|25.945472554950186|
|[0.04544,0.0,3.24...|19.79999924|21.280646033398444|
|[0.049320001,33.0...|28.20000076| 33.26874085034083|
|[0.055610001,70.0...|      

## Question 5: Evaluate the Linear Regression model by getting the RMSE and R-squared values

In [28]:
summary = model_lr.evaluate(test_df)

#### RMSE

In [29]:
print(summary.rootMeanSquaredError)

4.46829041377678


#### R-squared

In [30]:
print(summary.r2)

0.7668479296315855
